In [1]:
!pip install langchain-community pypdf chromadb faiss-cpu #faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00

In [2]:
def get_key():
  with open('20250723_OpenAI_API.key', 'r', encoding='utf-8') as file:
    return file.readline().rstrip()

In [3]:
import os
import urllib.request
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

os.environ['OPENAI_API_KEY'] = get_key()


In [4]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")


('2023_북한인권보고서.pdf', <http.client.HTTPMessage at 0x7a723cd4bc90>)

In [5]:
loader = PyPDFLoader('2023_북한인권보고서.pdf')
pages = loader.load_and_split()
print('청크의 수:', len(pages))


청크의 수: 445


In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)


In [19]:
splitted_docs = text_splitter.split_documents(pages)
print('분할된 청크의 수:', len(splitted_docs))


분할된 청크의 수: 496


In [8]:
chunks = [splitted_doc.page_content for splitted_doc in splitted_docs]
print('청크의 최대 길이 :',max(len(chunk) for chunk in chunks))
print('청크의 최소 길이 :',min(len(chunk) for chunk in chunks))
print('청크의 평균 길이 :',sum(map(len, chunks))/len(chunks))


청크의 최대 길이 : 1000
청크의 최소 길이 : 6
청크의 평균 길이 : 750.2983870967741


Chroma

In [9]:
db = Chroma.from_documents(splitted_docs, OpenAIEmbeddings(chunk_size=100))
print('문서의 수:', db._collection.count())


/tmp/ipython-input-9-3086761674.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = Chroma.from_documents(splitted_docs, OpenAIEmbeddings(chunk_size=100))


문서의 수: 496


In [10]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.7 MB/s eta 0:00:00


In [12]:
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(splitted_docs, OpenAIEmbeddings(chunk_size=100))
print('문서의 수:', db._collection.count())

문서의 수: 992


In [13]:
question = '북한의 교육과정'
docs = db.similarity_search(question)
print('문서의 수:', len(docs))


문서의 수: 4


In [14]:
for doc in docs:
  print(doc)
  print('--' * 10)


page_content='2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 또
한, 일반교육보다 정치사상교육을 앞세우고 있으며 교과과정에 실
탄사격을 하는 군사훈련을 편성하여 학생들을 의무적으로 참석하게 
하고 있다.
북한의 사회보장 제도로는 연로연금, 노동능력상실 연금, 유가족 
연금 등 생계가 결핍된 경우 기초적인 생계를 보장하기 위한 연금제
도가 있으며, 사회보험금의 성격을 지닌 보조금 제도가 있다. 연로' metadata={'source': '2023_북한인권보고서.pdf', 'trapped': '/False', 'creator': 'Adobe InDesign CS6 (Windows)', 'page_label': 

In [15]:
db_to_file = Chroma.from_documents(splitted_docs, OpenAIEmbeddings(chunk_size=100), persist_directory = './20250724_강의복습_벡터디비_chroma.db')
print('문서의 수:', db_to_file._collection.count())


문서의 수: 496


In [16]:
db_from_file = Chroma(persist_directory='./20250724_강의복습_벡터디비_chroma.db',
              embedding_function=OpenAIEmbeddings())
print('문서의 수:', db_from_file._collection.count())


문서의 수: 496


/tmp/ipython-input-16-2955316986.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db_from_file = Chroma(persist_directory='./20250724_강의복습_벡터디비_chroma.db',


In [17]:
question = '북한의 교육 과정'
top_three_docs = db_from_file.similarity_search_with_relevance_scores(question, k=3)

for doc in top_three_docs:
  print(doc)
  print('--' * 10)


(Document(metadata={'creator': 'Adobe InDesign CS6 (Windows)', 'moddate': '2023-07-31T13:57:54+09:00', 'trapped': '/False', 'total_pages': 448, 'page_label': '42', 'source': '2023_북한인권보고서.pdf', 'page': 41, 'producer': 'Adobe PDF Library 10.0.1', 'creationdate': '2023-07-31T13:50:27+09:00'}, page_content='2023 북한인권보고서\n40\n명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과\n서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교\n과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학\n년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교\n부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 \n수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 \n비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 \n의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 \n앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그\n만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육\n환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교\n육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제\n력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, \n학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 \n없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절\n히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모\n의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 \n제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다

Faiss

In [23]:
print(len(splitted_docs), sum(map(len, chunks)))

496 372148


In [25]:
new_total_len = 0
for idx, chunk in enumerate(chunks):
  new_total_len += len(chunk)
  if new_total_len > 300000:
    print("idx=", idx)
    break

idx= 399


In [26]:
embeddings = OpenAIEmbeddings(chunk_size=1000)

faiss_db = FAISS.from_documents(splitted_docs[:399], embeddings)
print('문서의 수:', faiss_db.index.ntotal)

문서의 수: 399


In [27]:
faiss_db.save_local('20250724_강의복습_벡터디비_faiss')

new_db_faiss = FAISS.load_local('20250724_강의복습_벡터디비_faiss',
                OpenAIEmbeddings(),
                allow_dangerous_deserialization=True)


In [28]:
question = '북한의 교육 과정'
docs = new_db_faiss.similarity_search(question)

for doc in docs:
  print(doc)
  print('--' * 10)


page_content='2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 또
한, 일반교육보다 정치사상교육을 앞세우고 있으며 교과과정에 실
탄사격을 하는 군사훈련을 편성하여 학생들을 의무적으로 참석하게 
하고 있다.
북한의 사회보장 제도로는 연로연금, 노동능력상실 연금, 유가족 
연금 등 생계가 결핍된 경우 기초적인 생계를 보장하기 위한 연금제
도가 있으며, 사회보험금의 성격을 지닌 보조금 제도가 있다. 연로' metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2023-07-